In [51]:
import os
import pdfplumber

def extract_text_from_all_pdfs(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(input_dir, file_name)
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    text += page.extract_text()
            output_file = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}.txt")
            with open(output_file, 'w+', encoding='utf-8') as f:
                f.write(text)

extract_text_from_all_pdfs('DSCI560_Lab5/', 'text_files/')



In [34]:
import os
import pdfplumber
import re
import requests

from bs4 import BeautifulSoup
def extract_text_from_all_pdfs(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    keys_encountered = set()
    patterns = {
                "operator": r"Well Operator : (.*?)\n",
                "well_name": r"Well or Facility Name : (.*?)\n",
                #"api":r"API#\s\d{2}-\d{3}-\d{5}",
                "enseco job#": r"\bJob (\d+)\b",
                "job_type": r"Type of Incident : (.*?)\n",
                "county": r"County : (.*?)\n",
                "latitude_pattern": r'(\d+°\d+\'\d+\.\d+\"[NS])',
                "longitude_pattern": r'(\d+°\d+\'\d+\.\d+\"[EW])',
                "datum": r"Vertical Datum to DDZ\s+([\d.]+ ft)"            
            }  
    results={}
    pattern = r"(Dato Stimulated Stimulated Formation Top \(Ft\) Bottom \(Ft\) Stimulation Stages Volume Volume Units)\s+([\s\S]+?)(?=\n\w)"
    matches = re.findall(pattern, text)
    for match in matches:
        value = match[1].strip()
        value_pattern=r"\d{2}/\w\s/\d{4}\s\d+\s(.*?)\s\d+\s\d+\s\d+\s\d+\s(.*?)"
        match = re.match(value_pattern, value)
        if match:
            l=value.split()
            results["Date Stimulated"]=''.join(l[0:2])
            results["Stimulated Information"]="".join(l[2:4])
            results["Top(Ft)"]=l[4]
            results["Bottom(Ft)"]=l[5]
            results["Stimulation Stages"]=l[6]
            results["Volume"]=l[7]
            results["Volume Units"]=l[8]

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            if key not in keys_encountered: 
                results[key] = match.group(1)
                keys_encountered.add(key)  
    return results
        
results=extract_text_from_all_pdfs('text_files/W23369.txt')
well_name=results["well_name"]
api=""
url1 = f"https://www.drillingedge.com/search?type=wells&well_name={well_name}&api_no={api}&operator_name=&lease_key=&state=&county=&section=&township=&range=&min_boe=&max_boe=&permit_start=&permit_end=&min_depth=&max_depth=&field_formation="

response = requests.get(url1)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    first_href = soup.find('table', class_='table wide-table interest_table').find('a')
    if first_href:
        url2 =first_href['href']
        response = requests.get(url2)
        print(url2)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            content = soup.find_all('p', class_='block_stat')
            for c in content:
                text= c.get_text()
                print(text)
            well_table = soup.find('article', class_='well_table')
            def get_data_by_th(soup, th_text):
                th = soup.find('th', string=th_text)
                if th and th.next_sibling:
                    return th.next_sibling.get_text(strip=True)
                return None
            if well_table:
                well_status = get_data_by_th(well_table, 'Well Status')
                well_type = get_data_by_th(well_table, 'Well Type')
                closest_city = get_data_by_th(well_table, 'Closest City')

                results["Well Status"]= well_status
                results["Well Type"]= well_type
                results["Closest City"]=closest_city
for key, value in results.items():
        print(f"{key}: {value}")

https://www.drillingedge.com/north-dakota/williams-county/wells/atlanta-14-6h/33-105-02719
280 Barrels of Oil Produced in Dec 2023
547 MCF of Gas Produced in Dec 2023
Date Stimulated: 11/E/2013
Stimulated Information: 3Forks
Top(Ft): 10913
Bottom(Ft): 20504
Stimulation Stages: 31
Volume: 51878
Volume Units: Barrels
operator: CONTINENTAL RESOURCES, INC.
well_name: ATLANTA 4-6H
enseco job#: 5284848
job_type: Tank Overflow
county: WILLIAMS
latitude_pattern: 48°06'33.667"N
longitude_pattern: 103°43'51.581"W
datum: 1967.00 ft
Well Status: Active
Well Type: Oil & Gas
Closest City: Williston
